# Tarea M6 MPAD de Zandalinas_Victor"

In [1]:
## 1. Introducción y Configuración
#### Importar pandas y las librerías utilizadas para resolver la tarea.

import sqlite3
import os
from datetime import datetime
import random
import math
import json

## 2. Análisis y Diseño"
# SISTEMA DE ANÁLISIS DE LANZAMIENTOS DE CORNERS EN FÚTBOL

## Descripción del Caso de Uso
El proyecto consiste en desarrollar una base de datos para analizar en detalle los lanzamientos de corners en fútbol profesional, con énfasis en los roles específicos de cada jugador, tanto en ataque como en defensa, sus movimientos y posicionamientos en el campo, y la efectividad de diferentes estrategias.

### 2.1 Descripción del Caso de Uso

1. Registro contextual de corners:
   - Información del partido, equipos, competición y datos generales del encuentro
   - Características básicas del corner (minuto, lado, lanzador)

2. Análisis de roles específicos de los jugadores:
   - En ataque: lanzador, rematador, bloqueador, arrastre, jugada en corto, rechace, cobertura
   - En defensa: marcaje zonal, al hombre, rechace o contrataque

3. Seguimiento de movimientos y coordenadas:
   - Posición inicial y final de cada jugador durante el corner
   - Coordenadas específicas del portero en defensa
   - Trayectoria del balón desde el lanzamiento hasta su destino final
   - Registro de lanzamientos indirectos (en corto + centro posterior)

4. Registro de resultados y métricas:
   - Desenlace del corner (gol, remate, despeje, etc.)
   - Valor de xG (gol esperado) generado en cada acción
   - Efectividad según tipos de lanzamiento y estrategias

5. Capacidad de análisis estadístico avanzado:
   - Patrones más efectivos por equipos y competiciones
   - Rendimiento individual de jugadores según roles
   - Comparativa de estrategias ofensivas y defensivas


## 2.2 Diseño de la Base de Datos

# Elección del Sistema de Gestión de Base de Datos

## Elección de SQLite como Sistema de Gestión de Base de Datos

Para este proyecto de análisis de corners en fútbol, hemos seleccionado SQLite como nuestro sistema de gestión de base de datos por las siguientes razones:

1. Portabilidad: SQLite almacena toda la base de datos en un único archivo, lo que facilita su distribución y uso en diferentes entornos sin necesidad de instalación de software adicional.

2. Simplicidad: No requiere configuración de servidor ni procesos de autenticación complejos, lo que agiliza el desarrollo y pruebas.

3. Rendimiento adecuado: Para el volumen de datos esperado en nuestro caso de uso (análisis de corners de varios equipos y competiciones), SQLite ofrece un rendimiento más que suficiente.

4. Soporte completo para SQL: SQLite implementa la mayoría de las características del estándar SQL, permitiendo realizar consultas complejas que necesitaremos para nuestros análisis.

5. Integración nativa con Python: A través del módulo sqlite3 de la biblioteca estándar de Python, la integración es directa y no requiere dependencias adicionales.

6. Transacciones ACID: SQLite garantiza que las operaciones de base de datos sean atómicas, consistentes, aisladas y duraderas, lo que es crucial para mantener la integridad de nuestros datos de análisis.

7. Escalabilidad adecuada: Aunque SQLite no está diseñado para aplicaciones con múltiples escrituras concurrentes, para nuestra aplicación de análisis deportivo (donde la mayoría de operaciones serán lecturas o inserciones controladas) resulta perfectamente adecuado.

SQLite nos permite enfocarnos en el diseño e implementación de la base de datos sin complicaciones adicionales de infraestructura, manteniendo toda la potencia analítica que necesitamos para nuestro caso de uso específico.


In [3]:
# Función para crear la base de datos y las tablas
def crear_base_datos():
    """
    Crea la base de datos SQLite y todas las tablas necesarias para el análisis de corners.
    """
    try:
        # Conectarse a la base de datos (la creará si no existe)
        conn = sqlite3.connect('corner_analysis.db')
        cursor = conn.cursor()
        
        # Tabla Competiciones
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS competiciones (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            pais TEXT NOT NULL,
            temporada TEXT NOT NULL,
            nivel TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Equipos
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS equipos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            ciudad TEXT,
            entrenador TEXT,
            estadio TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Jugadores
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS jugadores (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            equipo_id INTEGER,
            nombre TEXT NOT NULL,
            posicion TEXT,
            altura INTEGER,
            peso INTEGER,
            pie_dominante TEXT,
            fecha_nacimiento DATE,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (equipo_id) REFERENCES equipos (id)
        )
        ''')
        
        # Tabla Partidos
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS partidos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            competicion_id INTEGER,
            equipo_local_id INTEGER,
            equipo_visitante_id INTEGER,
            fecha DATE NOT NULL,
            resultado TEXT,
            estadio TEXT,
            asistencia INTEGER,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (competicion_id) REFERENCES competiciones (id),
            FOREIGN KEY (equipo_local_id) REFERENCES equipos (id),
            FOREIGN KEY (equipo_visitante_id) REFERENCES equipos (id)
        )
        ''')
        
        # Tabla Tipos de Lanzamiento
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS tipos_lanzamiento (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            descripcion TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Estrategias de Ataque
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS estrategias_ataque (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            descripcion TEXT,
            jugadores_area INTEGER,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Estrategias de Defensa
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS estrategias_defensa (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            descripcion TEXT,
            tipo TEXT CHECK(tipo IN ('zonal', 'individual', 'mixta')),
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Zonas de Destino
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS zonas_destino (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            descripcion TEXT,
            coordenadas TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Roles Ofensivos
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS roles_ofensivos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL CHECK(nombre IN ('lanzador', 'rematador', 'bloqueador', 'arrastre', 'corto', 'rechace', 'cobertura')),
            descripcion TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Roles Defensivos
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS roles_defensivos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL CHECK(nombre IN ('zonal', 'hombre', 'rechace', 'contrataque')),
            descripcion TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabla Corners
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS corners (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            partido_id INTEGER,
            equipo_id INTEGER,
            jugador_lanzador_id INTEGER,
            tipo_lanzamiento_id INTEGER,
            estrategia_ataque_id INTEGER,
            estrategia_defensa_id INTEGER,
            zona_destino_id INTEGER,
            minuto INTEGER,
            lado TEXT CHECK(lado IN ('izquierdo', 'derecho')),
            tiempo_extra BOOLEAN DEFAULT 0,
            jugadores_area_ataque INTEGER,
            jugadores_area_defensa INTEGER,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (partido_id) REFERENCES partidos (id),
            FOREIGN KEY (equipo_id) REFERENCES equipos (id),
            FOREIGN KEY (jugador_lanzador_id) REFERENCES jugadores (id),
            FOREIGN KEY (tipo_lanzamiento_id) REFERENCES tipos_lanzamiento (id),
            FOREIGN KEY (estrategia_ataque_id) REFERENCES estrategias_ataque (id),
            FOREIGN KEY (estrategia_defensa_id) REFERENCES estrategias_defensa (id),
            FOREIGN KEY (zona_destino_id) REFERENCES zonas_destino (id)
        )
        ''')
        
        # Tabla Participaciones de Jugadores
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS participaciones_jugador (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            corner_id INTEGER,
            jugador_id INTEGER,
            equipo_id INTEGER,
            rol_ofensivo_id INTEGER,
            rol_defensivo_id INTEGER,
            coord_inicial TEXT, -- formato: "x,y"
            coord_final TEXT, -- formato: "x,y"
            es_portero BOOLEAN DEFAULT 0,
            accion_realizada TEXT,
            resultado_accion TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (corner_id) REFERENCES corners (id),
            FOREIGN KEY (jugador_id) REFERENCES jugadores (id),
            FOREIGN KEY (equipo_id) REFERENCES equipos (id),
            FOREIGN KEY (rol_ofensivo_id) REFERENCES roles_ofensivos (id),
            FOREIGN KEY (rol_defensivo_id) REFERENCES roles_defensivos (id)
        )
        ''')
        
        # Tabla Trayectoria del Balón
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS trayectoria_balon (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            corner_id INTEGER,
            coord_lanzamiento TEXT, -- formato: "x,y"
            coord_destino_inicial TEXT, -- formato: "x,y"
            es_centro_indirecto BOOLEAN DEFAULT 0,
            coord_segundo_lanzamiento TEXT, -- formato: "x,y" (si aplica)
            coord_destino_final TEXT, -- formato: "x,y"
            velocidad FLOAT,
            altura FLOAT, -- altura máxima en metros
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (corner_id) REFERENCES corners (id)
        )
        ''')
        
        # Tabla Resultados de Corner
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS resultados_corner (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            corner_id INTEGER,
            tipo_resultado TEXT,
            gol BOOLEAN DEFAULT 0,
            remate BOOLEAN DEFAULT 0,
            jugador_remate_id INTEGER,
            xg_generado FLOAT,
            descripcion TEXT,
            fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (corner_id) REFERENCES corners (id),
            FOREIGN KEY (jugador_remate_id) REFERENCES jugadores (id)
        )
        ''')
        
        # Crear índices para optimizar consultas
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_corners_partido ON corners (partido_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_corners_equipo ON corners (equipo_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_corners_jugador ON corners (jugador_lanzador_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_participaciones_corner ON participaciones_jugador (corner_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_participaciones_jugador ON participaciones_jugador (jugador_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_participaciones_rol_of ON participaciones_jugador (rol_ofensivo_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_participaciones_rol_def ON participaciones_jugador (rol_defensivo_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_trayectoria_corner ON trayectoria_balon (corner_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_resultados_corner ON resultados_corner (corner_id)')
        
        conn.commit()
        print("Base de datos creada correctamente con todas las tablas necesarias")
        conn.close()
        return True
        
    except sqlite3.Error as e:
        print(f"Error al crear la base de datos: {e}")
        return False

# Ejecutamos la función para crear la base de datos
crear_base_datos()

Base de datos creada correctamente con todas las tablas necesarias


True

In [5]:
# Diagrama entidad-relación o similar. Puede ser una imagen o una descripción textual detallada

diagrama_er = """
# DIAGRAMA ENTIDAD-RELACIÓN: ANÁLISIS DE CORNERS EN FÚTBOL

## Entidades Principales:

1. COMPETICIONES (id, nombre, pais, temporada, nivel)
   - Almacena información sobre ligas y torneos

2. EQUIPOS (id, nombre, ciudad, entrenador, estadio)
   - Datos de los clubes de fútbol

3. JUGADORES (id, equipo_id, nombre, posicion, altura, peso, pie_dominante, fecha_nacimiento)
   - Información de los futbolistas

4. PARTIDOS (id, competicion_id, equipo_local_id, equipo_visitante_id, fecha, resultado, estadio, asistencia)
   - Registro de encuentros disputados

5. CORNERS (id, partido_id, equipo_id, jugador_lanzador_id, tipo_lanzamiento_id, estrategia_ataque_id, estrategia_defensa_id, zona_destino_id, minuto, lado, tiempo_extra, jugadores_area_ataque, jugadores_area_defensa)
   - Registro central de cada lanzamiento de corner

6. TIPOS_LANZAMIENTO (id, nombre, descripcion)
   - Catálogo de formas de lanzar corners

7. ESTRATEGIAS_ATAQUE (id, nombre, descripcion, jugadores_area)
   - Catálogo de patrones ofensivos en corners

8. ESTRATEGIAS_DEFENSA (id, nombre, descripcion, tipo)
   - Catálogo de sistemas defensivos en corners

9. ZONAS_DESTINO (id, nombre, descripcion, coordenadas)
   - Áreas del campo donde se dirige el balón

10. ROLES_OFENSIVOS (id, nombre, descripcion)
    - Catálogo de roles en ataque (lanzador, rematador, bloqueador, arrastre, corto, rechace, cobertura)

11. ROLES_DEFENSIVOS (id, nombre, descripcion)
    - Catálogo de roles en defensa (zonal, hombre, rechace, contraataque)

12. PARTICIPACIONES_JUGADOR (id, corner_id, jugador_id, equipo_id, rol_ofensivo_id, rol_defensivo_id, coord_inicial, coord_final, es_portero, accion_realizada, resultado_accion)
    - Registro detallado de la participación de cada jugador en un corner

13. TRAYECTORIA_BALON (id, corner_id, coord_lanzamiento, coord_destino_inicial, es_centro_indirecto, coord_segundo_lanzamiento, coord_destino_final, velocidad, altura)
    - Seguimiento de la trayectoria del balón en cada corner

14. RESULTADOS_CORNER (id, corner_id, tipo_resultado, gol, remate, jugador_remate_id, xg_generado, descripcion)
    - Registro del desenlace de cada corner

## Relaciones Principales:
- COMPETICIONES 1:N PARTIDOS
- EQUIPOS 1:N JUGADORES
- EQUIPOS 1:N PARTIDOS
- PARTIDOS 1:N CORNERS
- CORNERS 1:N PARTICIPACIONES_JUGADOR
- JUGADORES 1:N PARTICIPACIONES_JUGADOR
- CORNERS 1:1 TRAYECTORIA_BALON
- CORNERS 1:1 RESULTADOS_CORNER
- ROLES_OFENSIVOS 1:N PARTICIPACIONES_JUGADOR
- ROLES_DEFENSIVOS 1:N PARTICIPACIONES_JUGADOR
"""

print(diagrama_er)


# DIAGRAMA ENTIDAD-RELACIÓN: ANÁLISIS DE CORNERS EN FÚTBOL

## Entidades Principales:

1. COMPETICIONES (id, nombre, pais, temporada, nivel)
   - Almacena información sobre ligas y torneos

2. EQUIPOS (id, nombre, ciudad, entrenador, estadio)
   - Datos de los clubes de fútbol

3. JUGADORES (id, equipo_id, nombre, posicion, altura, peso, pie_dominante, fecha_nacimiento)
   - Información de los futbolistas

4. PARTIDOS (id, competicion_id, equipo_local_id, equipo_visitante_id, fecha, resultado, estadio, asistencia)
   - Registro de encuentros disputados

5. CORNERS (id, partido_id, equipo_id, jugador_lanzador_id, tipo_lanzamiento_id, estrategia_ataque_id, estrategia_defensa_id, zona_destino_id, minuto, lado, tiempo_extra, jugadores_area_ataque, jugadores_area_defensa)
   - Registro central de cada lanzamiento de corner

6. TIPOS_LANZAMIENTO (id, nombre, descripcion)
   - Catálogo de formas de lanzar corners

7. ESTRATEGIAS_ATAQUE (id, nombre, descripcion, jugadores_area)
   - Catálogo 

In [8]:
## 3.2 Implementación de Funciones CRUD

# Clase principal para gestionar la base de datos
class CornerAnalysisDB:
    def __init__(self, db_name='corner_analysis.db'):
        """Inicializa la conexión a la base de datos."""
        self.db_name = db_name
        self.conn = None
        self.cursor = None
    
    def connect(self):
        """Establece conexión con la base de datos."""
        try:
            self.conn = sqlite3.connect(self.db_name)
            self.cursor = self.conn.cursor()
            print(f"Conexión establecida con {self.db_name}")
            return True
        except sqlite3.Error as e:
            print(f"Error al conectar a la base de datos: {e}")
            return False
    
    def close(self):
        """Cierra la conexión con la base de datos."""
        if self.conn:
            self.conn.close()
            print("Conexión cerrada")
    
    # Funciones CRUD para Competiciones
    def crear_competicion(self, nombre, pais, temporada, nivel=None):
        """Crea una nueva competición en la base de datos."""
        try:
            self.cursor.execute('''
            INSERT INTO competiciones (nombre, pais, temporada, nivel)
            VALUES (?, ?, ?, ?)
            ''', (nombre, pais, temporada, nivel))
            self.conn.commit()
            return self.cursor.lastrowid
        except sqlite3.Error as e:
            print(f"Error al crear competición: {e}")
            return None
    
    def obtener_competicion(self, competicion_id):
        """Obtiene una competición por su ID."""
        try:
            self.cursor.execute('SELECT * FROM competiciones WHERE id = ?', (competicion_id,))
            return self.cursor.fetchone()
        except sqlite3.Error as e:
            print(f"Error al obtener competición: {e}")
            return None
    
    def actualizar_competicion(self, competicion_id, nombre=None, pais=None, temporada=None, nivel=None):
        """Actualiza la información de una competición."""
        try:
            # Obtener datos actuales
            self.cursor.execute('SELECT nombre, pais, temporada, nivel FROM competiciones WHERE id = ?', (competicion_id,))
            comp_actual = self.cursor.fetchone()
            
            if not comp_actual:
                print(f"No se encontró la competición con ID {competicion_id}")
                return False
            
            # Usar valores actuales si no se proporcionan nuevos
            nombre = nombre if nombre is not None else comp_actual[0]
            pais = pais if pais is not None else comp_actual[1]
            temporada = temporada if temporada is not None else comp_actual[2]
            nivel = nivel if nivel is not None else comp_actual[3]
            
            self.cursor.execute('''
            UPDATE competiciones 
            SET nombre = ?, pais = ?, temporada = ?, nivel = ?
            WHERE id = ?
            ''', (nombre, pais, temporada, nivel, competicion_id))
            self.conn.commit()
            return True
        except sqlite3.Error as e:
            print(f"Error al actualizar competición: {e}")
            return False
    
    def eliminar_competicion(self, competicion_id):
        """Elimina una competición de la base de datos."""
        try:
            self.cursor.execute('DELETE FROM competiciones WHERE id = ?', (competicion_id,))
            self.conn.commit()
            return True
        except sqlite3.Error as e:
            print(f"Error al eliminar competición: {e}")
            return False
    
    # Funciones CRUD para Equipos - AÑADIDAS
    def crear_equipo(self, nombre, ciudad=None, entrenador=None, estadio=None):
        """Crea un nuevo equipo en la base de datos."""
        try:
            self.cursor.execute('''
            INSERT INTO equipos (nombre, ciudad, entrenador, estadio)
            VALUES (?, ?, ?, ?)
            ''', (nombre, ciudad, entrenador, estadio))
            self.conn.commit()
            return self.cursor.lastrowid
        except sqlite3.Error as e:
            print(f"Error al crear equipo: {e}")
            return None
    
    def obtener_equipo(self, equipo_id):
        """Obtiene un equipo por su ID."""
        try:
            self.cursor.execute('SELECT * FROM equipos WHERE id = ?', (equipo_id,))
            return self.cursor.fetchone()
        except sqlite3.Error as e:
            print(f"Error al obtener equipo: {e}")
            return None
    
    def actualizar_equipo(self, equipo_id, nombre=None, ciudad=None, entrenador=None, estadio=None):
        """Actualiza la información de un equipo."""
        try:
            # Obtener datos actuales
            self.cursor.execute('SELECT nombre, ciudad, entrenador, estadio FROM equipos WHERE id = ?', (equipo_id,))
            equipo_actual = self.cursor.fetchone()
            
            if not equipo_actual:
                print(f"No se encontró el equipo con ID {equipo_id}")
                return False
            
            # Usar valores actuales si no se proporcionan nuevos
            nombre = nombre if nombre is not None else equipo_actual[0]
            ciudad = ciudad if ciudad is not None else equipo_actual[1]
            entrenador = entrenador if entrenador is not None else equipo_actual[2]
            estadio = estadio if estadio is not None else equipo_actual[3]
            
            self.cursor.execute('''
            UPDATE equipos 
            SET nombre = ?, ciudad = ?, entrenador = ?, estadio = ?
            WHERE id = ?
            ''', (nombre, ciudad, entrenador, estadio, equipo_id))
            self.conn.commit()
            return True
        except sqlite3.Error as e:
            print(f"Error al actualizar equipo: {e}")
            return False
    
    def eliminar_equipo(self, equipo_id):
        """Elimina un equipo de la base de datos."""
        try:
            self.cursor.execute('DELETE FROM equipos WHERE id = ?', (equipo_id,))
            self.conn.commit()
            return True
        except sqlite3.Error as e:
            print(f"Error al eliminar equipo: {e}")
            return False
    
    # Funciones CRUD para Jugadores - AÑADIDAS
    def crear_jugador(self, equipo_id, nombre, posicion=None, altura=None, 
                     peso=None, pie_dominante=None, fecha_nacimiento=None):
        """Crea un nuevo jugador en la base de datos."""
        try:
            self.cursor.execute('''
            INSERT INTO jugadores (equipo_id, nombre, posicion, altura, peso, 
                                  pie_dominante, fecha_nacimiento)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (equipo_id, nombre, posicion, altura, peso, 
                 pie_dominante, fecha_nacimiento))
            self.conn.commit()
            return self.cursor.lastrowid
        except sqlite3.Error as e:
            print(f"Error al crear jugador: {e}")
            return None
    
    def obtener_jugador(self, jugador_id):
        """Obtiene un jugador por su ID."""
        try:
            self.cursor.execute('SELECT * FROM jugadores WHERE id = ?', (jugador_id,))
            return self.cursor.fetchone()
        except sqlite3.Error as e:
            print(f"Error al obtener jugador: {e}")
            return None
    
    # Funciones CRUD para Corners
    def crear_corner(self, partido_id, equipo_id, jugador_lanzador_id, tipo_lanzamiento_id, 
                    estrategia_ataque_id, estrategia_defensa_id, zona_destino_id,
                    minuto, lado, tiempo_extra=0, jugadores_area_ataque=None, jugadores_area_defensa=None):
        """Registra un nuevo lanzamiento de corner."""
        try:
            self.cursor.execute('''
            INSERT INTO corners (
                partido_id, equipo_id, jugador_lanzador_id, tipo_lanzamiento_id,
                estrategia_ataque_id, estrategia_defensa_id, zona_destino_id,
                minuto, lado, tiempo_extra, jugadores_area_ataque, jugadores_area_defensa
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                partido_id, equipo_id, jugador_lanzador_id, tipo_lanzamiento_id,
                estrategia_ataque_id, estrategia_defensa_id, zona_destino_id,
                minuto, lado, tiempo_extra, jugadores_area_ataque, jugadores_area_defensa
            ))
            self.conn.commit()
            return self.cursor.lastrowid
        except sqlite3.Error as e:
            print(f"Error al registrar corner: {e}")
            return None
    
    def obtener_corner(self, corner_id):
        """Obtiene los detalles de un corner por su ID."""
        try:
            self.cursor.execute('''
            SELECT 
                c.*,
                p.fecha AS fecha_partido,
                e.nombre AS equipo_nombre,
                j.nombre AS jugador_nombre,
                tl.nombre AS tipo_lanzamiento,
                ea.nombre AS estrategia_ataque,
                ed.nombre AS estrategia_defensa,
                zd.nombre AS zona_destino
            FROM corners c
            JOIN partidos p ON c.partido_id = p.id
            JOIN equipos e ON c.equipo_id = e.id
            JOIN jugadores j ON c.jugador_lanzador_id = j.id
            JOIN tipos_lanzamiento tl ON c.tipo_lanzamiento_id = tl.id
            JOIN estrategias_ataque ea ON c.estrategia_ataque_id = ea.id
            JOIN estrategias_defensa ed ON c.estrategia_defensa_id = ed.id
            JOIN zonas_destino zd ON c.zona_destino_id = zd.id
            WHERE c.id = ?
            ''', (corner_id,))
            return self.cursor.fetchone()
        except sqlite3.Error as e:
            print(f"Error al obtener corner: {e}")
            return None
    
    def actualizar_corner(self, corner_id, **kwargs):
        """Actualiza la información de un corner."""
        try:
            # Verificar que el corner existe
            self.cursor.execute('SELECT id FROM corners WHERE id = ?', (corner_id,))
            if not self.cursor.fetchone():
                print(f"No se encontró el corner con ID {corner_id}")
                return False
            
            # Construir la consulta de actualización dinámicamente
            campos = []
            valores = []
            
            for campo, valor in kwargs.items():
                if valor is not None:  # Solo actualizar campos con valores proporcionados
                    campos.append(f"{campo} = ?")
                    valores.append(valor)
            
            if not campos:  # Si no hay campos para actualizar
                return True
            
            query = f"UPDATE corners SET {', '.join(campos)} WHERE id = ?"
            valores.append(corner_id)
            
            self.cursor.execute(query, tuple(valores))
            self.conn.commit()
            return True
        except sqlite3.Error as e:
            print(f"Error al actualizar corner: {e}")
            return False
    
    def eliminar_corner(self, corner_id):
        """Elimina un corner y sus datos relacionados de la base de datos."""
        try:
            # Iniciar transacción
            self.conn.execute("BEGIN TRANSACTION")
            
            # Eliminar registros relacionados en las tablas dependientes
            self.cursor.execute('DELETE FROM resultados_corner WHERE corner_id = ?', (corner_id,))
            self.cursor.execute('DELETE FROM trayectoria_balon WHERE corner_id = ?', (corner_id,))
            self.cursor.execute('DELETE FROM participaciones_jugador WHERE corner_id = ?', (corner_id,))
            
            # Finalmente eliminar el corner
            self.cursor.execute('DELETE FROM corners WHERE id = ?', (corner_id,))
            
            # Confirmar la transacción
            self.conn.commit()
            return True
        except sqlite3.Error as e:
            # Revertir la transacción en caso de error
            self.conn.rollback()
            print(f"Error al eliminar corner: {e}")
            return False
    
    # Función para registrar participación de jugador
    def registrar_participacion_jugador(self, corner_id, jugador_id, equipo_id, 
                                       rol_ofensivo_id=None, rol_defensivo_id=None,
                                       coord_inicial=None, coord_final=None, 
                                       es_portero=0, accion_realizada=None, resultado_accion=None):
        """Registra la participación de un jugador en un corner específico."""
        try:
            self.cursor.execute('''
            INSERT INTO participaciones_jugador (
                corner_id, jugador_id, equipo_id, rol_ofensivo_id, rol_defensivo_id,
                coord_inicial, coord_final, es_portero, accion_realizada, resultado_accion
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                corner_id, jugador_id, equipo_id, rol_ofensivo_id, rol_defensivo_id,
                coord_inicial, coord_final, es_portero, accion_realizada, resultado_accion
            ))
            self.conn.commit()
            return self.cursor.lastrowid
        except sqlite3.Error as e:
            print(f"Error al registrar participación de jugador: {e}")
            return None
    
    # Función para registrar resultado de corner
    def registrar_resultado_corner(self, corner_id, tipo_resultado, gol=0, remate=0, 
                                  jugador_remate_id=None, xg_generado=0.0, descripcion=None):
        """Registra el resultado de un lanzamiento de corner."""
        try:
            self.cursor.execute('''
            INSERT INTO resultados_corner (
                corner_id, tipo_resultado, gol, remate, jugador_remate_id, xg_generado, descripcion
            )
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (corner_id, tipo_resultado, gol, remate, jugador_remate_id, xg_generado, descripcion))
            self.conn.commit()
            return self.cursor.lastrowid
        except sqlite3.Error as e:
            print(f"Error al registrar resultado: {e}")
            return None

# Demostración de uso de las funciones CRUD
def demostrar_funciones_crud():
    """Demuestra el uso de las funciones CRUD con ejemplos prácticos."""
    try:
        # Crear una instancia de la clase de base de datos
        db = CornerAnalysisDB()
        db.connect()
        
        # Ejemplo 1: Crear y obtener una competición
        print("\nEjemplo 1: Crear y obtener una competición")
        comp_id = db.crear_competicion("LaLiga", "España", "2023-2024", "Primera División")
        if comp_id:
            print(f"Competición creada con ID: {comp_id}")
            comp_data = db.obtener_competicion(comp_id)
            print(f"Datos de la competición: {comp_data}")
        
        # Ejemplo 2: Actualizar una competición
        print("\nEjemplo 2: Actualizar una competición")
        if db.actualizar_competicion(comp_id, nivel="Primera División - Profesional"):
            print(f"Competición actualizada correctamente")
            comp_data = db.obtener_competicion(comp_id)
            print(f"Datos actualizados: {comp_data}")
        
        # Ejemplo 3: Operaciones más complejas (creación de un corner completo)
        print("\nEjemplo 3: Creación de un corner completo con sus relaciones")
        
        # Crear entidades relacionadas necesarias
        equipo_id = db.crear_equipo("FC Barcelona", "Barcelona", "Xavi Hernández", "Camp Nou")
        jugador_id = db.crear_jugador(equipo_id, "Leo Messi", "Delantero", 170, 72, "Izquierdo", "1987-06-24")
        tipo_id = db.cursor.execute("INSERT INTO tipos_lanzamiento (nombre, descripcion) VALUES (?, ?)", 
                                   ("Cerrado", "Balón con efecto hacia la portería")).lastrowid
        db.conn.commit()
        
        # Más entidades relacionadas
        estrategia_ataque_id = db.cursor.execute("INSERT INTO estrategias_ataque (nombre, descripcion, jugadores_area) VALUES (?, ?, ?)", 
                                               ("Bloqueo y remate", "Jugador bloquea para liberar al rematador", 5)).lastrowid
        estrategia_defensa_id = db.cursor.execute("INSERT INTO estrategias_defensa (nombre, descripcion, tipo) VALUES (?, ?, ?)", 
                                                ("Mixta con marcaje individual", "Combinación de zonas y marcaje individual", "mixta")).lastrowid
        zona_destino_id = db.cursor.execute("INSERT INTO zonas_destino (nombre, descripcion, coordenadas) VALUES (?, ?, ?)", 
                                          ("Primer palo", "Zona cercana al primer poste", "5,2")).lastrowid
        db.conn.commit()
        
        # Crear un partido
        partido_id = db.cursor.execute('''INSERT INTO partidos 
                                        (competicion_id, equipo_local_id, equipo_visitante_id, fecha, resultado, estadio) 
                                        VALUES (?, ?, ?, ?, ?, ?)''', 
                                      (comp_id, equipo_id, equipo_id, "2023-09-16", "2-1", "Camp Nou")).lastrowid
        db.conn.commit()
        
        # Registrar el corner
        corner_id = db.crear_corner(
            partido_id, equipo_id, jugador_id, tipo_id, 
            estrategia_ataque_id, estrategia_defensa_id, zona_destino_id,
            65, "derecho", 0, 5, 6
        )
        
        if corner_id:
            print(f"Corner registrado con ID: {corner_id}")
            
            # Registrar participación de un jugador
            participacion_id = db.registrar_participacion_jugador(
                corner_id, jugador_id, equipo_id, 
                rol_ofensivo_id=db.cursor.execute("INSERT INTO roles_ofensivos (nombre, descripcion) VALUES (?, ?)", 
                                                ("rematador", "Jugador que remata el balón")).lastrowid,
                coord_inicial="10,15", coord_final="5,2", 
                accion_realizada="Remate de cabeza", resultado_accion="Gol"
            )
            db.conn.commit()
            
            # Registrar el resultado
            resultado_id = db.registrar_resultado_corner(
                corner_id, "Gol", 1, 1, jugador_id, 0.23, 
                "Remate de cabeza al primer palo, el portero no llega a tiempo"
            )
            
            print(f"Participación registrada con ID: {participacion_id}")
            print(f"Resultado registrado con ID: {resultado_id}")
            
            # Obtener detalles completos del corner
            corner_detalle = db.obtener_corner(corner_id)
            print(f"Detalles del corner: {corner_detalle}")
        
        # Cerrar la conexión
        db.close()
        return True
    
    except Exception as e:
        print(f"Error en la demostración: {e}")
        return False

# Ejecutar la demostración
demostrar_funciones_crud()

Conexión establecida con corner_analysis.db

Ejemplo 1: Crear y obtener una competición
Competición creada con ID: 3
Datos de la competición: (3, 'LaLiga', 'España', '2023-2024', 'Primera División', '2025-03-21 11:47:47')

Ejemplo 2: Actualizar una competición
Competición actualizada correctamente
Datos actualizados: (3, 'LaLiga', 'España', '2023-2024', 'Primera División - Profesional', '2025-03-21 11:47:47')

Ejemplo 3: Creación de un corner completo con sus relaciones
Corner registrado con ID: 2
Participación registrada con ID: 2
Resultado registrado con ID: 2
Detalles del corner: (2, 2, 2, 2, 2, 2, 2, 2, 65, 'derecho', 0, 5, 6, '2025-03-21 11:47:47', '2023-09-16', 'FC Barcelona', 'Leo Messi', 'Cerrado', 'Bloqueo y remate', 'Mixta con marcaje individual', 'Primer palo')
Conexión cerrada


True

In [9]:
## 4. Limpieza y Preprocesamiento de Datos

# Función para importar datos de muestra
def importar_datos_muestra():
    """
    Importa un conjunto de datos de muestra para el análisis de corners.
    Genera datos sintéticos que simulan la recolección de información de corners
    en partidos reales.
    """
    try:
        # Conexión a la base de datos
        db = CornerAnalysisDB()
        db.connect()
        
        print("Importando datos de muestra para análisis de corners...")
        
        # 1. Datos de competiciones
        competiciones = [
            ('LaLiga', 'España', '2023-2024', 'Primera División'),
            ('Premier League', 'Inglaterra', '2023-2024', 'Primera División'),
            ('Serie A', 'Italia', '2023-2024', 'Primera División'),
            ('Bundesliga', 'Alemania', '2023-2024', 'Primera División'),
            ('Ligue 1', 'Francia', '2023-2024', 'Primera División')
        ]
        
        competicion_ids = []
        for comp in competiciones:
            comp_id = db.crear_competicion(*comp)
            if comp_id:
                competicion_ids.append(comp_id)
        
        print(f"  - {len(competicion_ids)} competiciones importadas")
        
        # 2. Datos de equipos
        equipos = [
            # LaLiga
            ('Real Madrid', 'Madrid', 'Carlo Ancelotti', 'Santiago Bernabéu'),
            ('FC Barcelona', 'Barcelona', 'Xavi Hernández', 'Camp Nou'),
            ('Atlético Madrid', 'Madrid', 'Diego Simeone', 'Metropolitano'),
            ('Sevilla FC', 'Sevilla', 'Quique Sánchez Flores', 'Ramón Sánchez-Pizjuán'),
            # Premier League
            ('Manchester City', 'Manchester', 'Pep Guardiola', 'Etihad Stadium'),
            ('Liverpool FC', 'Liverpool', 'Jürgen Klopp', 'Anfield'),
            ('Arsenal FC', 'Londres', 'Mikel Arteta', 'Emirates Stadium'),
            ('Manchester United', 'Manchester', 'Erik ten Hag', 'Old Trafford')
        ]
        
        equipo_ids = []
        for equipo in equipos:
            equipo_id = db.crear_equipo(*equipo)
            if equipo_id:
                equipo_ids.append(equipo_id)
        
        print(f"  - {len(equipo_ids)} equipos importados")
        
        # 3. Datos de jugadores
        jugadores = []
        posiciones = ['Portero', 'Defensa Central', 'Lateral Derecho', 'Lateral Izquierdo', 
                     'Mediocentro', 'Interior Derecho', 'Interior Izquierdo', 'Mediapunta', 
                     'Extremo Derecho', 'Extremo Izquierdo', 'Delantero Centro']
        pies = ['Derecho', 'Izquierdo', 'Ambidiestro']
        
        nombres = [
            'Carlos Rodríguez', 'Juan Martínez', 'Miguel López', 'David García', 
            'Fernando Pérez', 'Javier Sánchez', 'Antonio Gómez', 'José Fernández',
            'John Smith', 'Michael Johnson', 'Robert Williams', 'David Brown',
            'Marco Rossi', 'Andrea Bianchi', 'Giuseppe Verdi', 'Alessandro Ferrari',
            'Hans Müller', 'Thomas Weber', 'Klaus Schmidt', 'Wolfgang Becker'
        ]
        
        jugador_ids = []
        
        # Generar jugadores para cada equipo
        for equipo_id in equipo_ids:
            # Generar entre 15-20 jugadores por equipo
            for i in range(random.randint(15, 20)):
                nombre = random.choice(nombres)
                posicion = random.choice(posiciones)
                altura = random.randint(165, 200)  # en cm
                peso = random.randint(60, 95)  # en kg
                pie_dominante = random.choice(pies)
                
                # Fecha de nacimiento (jugadores entre 18 y 38 años)
                years_ago = random.randint(18, 38)
                fecha_nacimiento = datetime.now() - timedelta(days=365 * years_ago + random.randint(0, 364))
                fecha_nacimiento_str = fecha_nacimiento.strftime('%Y-%m-%d')
                
                jugador_id = db.crear_jugador(equipo_id, nombre, posicion, altura, peso, pie_dominante, fecha_nacimiento_str)
                if jugador_id:
                    jugador_ids.append(jugador_id)
                    jugadores.append((jugador_id, nombre, equipo_id, posicion))
        
        print(f"  - {len(jugador_ids)} jugadores importados")
        
        # 4. Catálogos de referencia
        # Tipos de lanzamiento
        tipos_lanzamiento = [
            ('Cerrado', 'Balón con efecto hacia la portería'),
            ('Abierto', 'Balón con efecto alejándose de la portería'),
            ('Raso', 'Balón a ras de suelo'),
            ('En corto', 'Pase corto a un compañero cercano'),
            ('Directo', 'Lanzamiento directo al área')
        ]
        
        tipo_ids = []
        for tipo in tipos_lanzamiento:
            db.cursor.execute("INSERT INTO tipos_lanzamiento (nombre, descripcion) VALUES (?, ?)", tipo)
            tipo_ids.append(db.cursor.lastrowid)
        
        # Estrategias de ataque
        estrategias_ataque = [
            ('Bloqueo y remate', 'Jugadores bloquean para liberar al rematador', 5),
            ('Desmarque al primer palo', 'Movimiento rápido al primer palo', 4),
            ('Centro al segundo palo', 'Buscar altura en el segundo palo', 3),
            ('Jugada ensayada', 'Movimientos coordinados previos al lanzamiento', 6),
            ('Arrastre y espacio', 'Generar espacio con movimientos de arrastre', 5)
        ]
        
        estrategia_ataque_ids = []
        for est in estrategias_ataque:
            db.cursor.execute("INSERT INTO estrategias_ataque (nombre, descripcion, jugadores_area) VALUES (?, ?, ?)", est)
            estrategia_ataque_ids.append(db.cursor.lastrowid)
        
        # Estrategias de defensa
        estrategias_defensa = [
            ('Marcaje zonal', 'Defensores cubren zonas predefinidas', 'zonal'),
            ('Marcaje individual', 'Cada defensor marca a un atacante específico', 'individual'),
            ('Mixta básica', 'Combinación de zonas y marcajes individuales', 'mixta'),
            ('Zona con referencias', 'Zonas con ajustes según movimientos ofensivos', 'zonal'),
            ('Hombre a hombre con ayudas', 'Marcaje individual con coberturas', 'individual')
        ]
        
        estrategia_defensa_ids = []
        for est in estrategias_defensa:
            db.cursor.execute("INSERT INTO estrategias_defensa (nombre, descripcion, tipo) VALUES (?, ?, ?)", est)
            estrategia_defensa_ids.append(db.cursor.lastrowid)
        
        # Zonas de destino
        zonas_destino = [
            ('Primer palo', 'Zona cercana al primer poste', '5,2'),
            ('Segundo palo', 'Zona cercana al segundo poste', '16,2'),
            ('Punto de penalti', 'Zona central del área', '11,6'),
            ('Frontal del área', 'Borde del área', '11,18'),
            ('Centro del área', 'Zona central entre los postes', '11,3')
        ]
        
        zona_destino_ids = []
        for zona in zonas_destino:
            db.cursor.execute("INSERT INTO zonas_destino (nombre, descripcion, coordenadas) VALUES (?, ?, ?)", zona)
            zona_destino_ids.append(db.cursor.lastrowid)
        
        # Roles ofensivos
        roles_ofensivos = [
            ('lanzador', 'Jugador que ejecuta el corner'),
            ('rematador', 'Jugador que intenta rematar el balón'),
            ('bloqueador', 'Jugador que bloquea defensores'),
            ('arrastre', 'Jugador que arrastra marcas'),
            ('corto', 'Jugador que recibe en corto'),
            ('rechace', 'Jugador posicionado para segundas jugadas'),
            ('cobertura', 'Jugador que da equilibrio defensivo')
        ]
        
        rol_ofensivo_ids = []
        for rol in roles_ofensivos:
            db.cursor.execute("INSERT INTO roles_ofensivos (nombre, descripcion) VALUES (?, ?)", rol)
            rol_ofensivo_ids.append(db.cursor.lastrowid)
        
        # Roles defensivos
        roles_defensivos = [
            ('zonal', 'Defensor en zona específica'),
            ('hombre', 'Defensor en marcaje individual'),
            ('rechace', 'Defensor para ganar segundas jugadas'),
            ('contrataque', 'Defensor posicionado para contrataque')
        ]
        
        rol_defensivo_ids = []
        for rol in roles_defensivos:
            db.cursor.execute("INSERT INTO roles_defensivos (nombre, descripcion) VALUES (?, ?)", rol)
            rol_defensivo_ids.append(db.cursor.lastrowid)
        
        db.conn.commit()
        print("  - Catálogos de referencia importados correctamente")
        
        # 5. Generar partidos y corners de muestra
        partidos_generados = 0
        corners_generados = 0
        
        # Generar 20 partidos de muestra
        for _ in range(20):
            # Seleccionar competición y equipos al azar
            comp_id = random.choice(competicion_ids)
            local_id = random.choice(equipo_ids)
            visitante_id = random.choice([e for e in equipo_ids if e != local_id])
            
            # Fecha aleatoria en la temporada actual
            dias_atras = random.randint(1, 180)
            fecha_partido = datetime.now() - timedelta(days=dias_atras)
            fecha_str = fecha_partido.strftime('%Y-%m-%d')
            
            # Resultado aleatorio
            goles_local = random.randint(0, 4)
            goles_visitante = random.randint(0, 3)
            resultado = f"{goles_local}-{goles_visitante}"
            
            # Estadio y asistencia
            estadio = db.cursor.execute("SELECT estadio FROM equipos WHERE id = ?", (local_id,)).fetchone()[0]
            asistencia = random.randint(30000, 80000)
            
            # Insertar partido
            db.cursor.execute('''
            INSERT INTO partidos (competicion_id, equipo_local_id, equipo_visitante_id, fecha, resultado, estadio, asistencia)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (comp_id, local_id, visitante_id, fecha_str, resultado, estadio, asistencia))
            
            partido_id = db.cursor.lastrowid
            partidos_generados += 1
            
            # Generar corners para este partido (3-12 corners)
            num_corners = random.randint(3, 12)
            
            for _ in range(num_corners):
                # Determinar equipo atacante
                equipo_id = random.choice([local_id, visitante_id])
                
                # Seleccionar jugador lanzador (buscar un extremo o lateral de ese equipo)
                jugadores_equipo = [j for j in jugadores if j[2] == equipo_id]
                lanzadores_potenciales = [j for j in jugadores_equipo if 'Lateral' in j[3] or 'Extremo' in j[3]]
                if not lanzadores_potenciales:
                    lanzadores_potenciales = jugadores_equipo
                
                lanzador_id = random.choice(lanzadores_potenciales)[0]
                
                # Datos aleatorios del corner
                minuto = random.randint(1, 90)
                lado = random.choice(['izquierdo', 'derecho'])
                tipo_lanzamiento_id = random.choice(tipo_ids)
                estrategia_ataque_id = random.choice(estrategia_ataque_ids)
                estrategia_defensa_id = random.choice(estrategia_defensa_ids)
                zona_destino_id = random.choice(zona_destino_ids)
                tiempo_extra = 1 if minuto > 45 and random.random() < 0.1 else 0
                jugadores_area_ataque = random.randint(4, 7)
                jugadores_area_defensa = random.randint(5, 8)
                
                # Registrar corner
                corner_id = db.crear_corner(
                    partido_id, equipo_id, lanzador_id, tipo_lanzamiento_id,
                    estrategia_ataque_id, estrategia_defensa_id, zona_destino_id,
                    minuto, lado, tiempo_extra, jugadores_area_ataque, jugadores_area_defensa
                )
                
                if corner_id:
                    corners_generados += 1
                    
                    # Registrar trayectoria del balón
                    es_centro_indirecto = random.random() < 0.25  # 25% de probabilidad de ser un corner en corto
                    
                    # Coordenadas según el lado
                    if lado == 'izquierdo':
                        coord_lanzamiento = "0,0"
                    else:
                        coord_lanzamiento = "0,40"  # Suponiendo campo de 40 unidades de ancho
                    
                    # Punto inicial de destino
                    if es_centro_indirecto:
                        # Corner en corto
                        coord_destino_inicial = f"{random.randint(3, 8)},{random.randint(5, 15) if lado == 'izquierdo' else random.randint(25, 35)}"
                        coord_segundo_lanzamiento = coord_destino_inicial
                        # Segundo centro
                        coord_destino_final = f"{random.randint(5, 16)},{random.randint(2, 10)}"
                    else:
                        # Corner directo
                        coord_destino_inicial = f"{random.randint(5, 16)},{random.randint(2, 10)}"
                        coord_segundo_lanzamiento = None
                        coord_destino_final = coord_destino_inicial
                    
                    velocidad = random.uniform(20.0, 35.0)  # m/s
                    altura = random.uniform(1.5, 4.0)  # metros
                    
                    db.registrar_trayectoria_balon(
                        corner_id, coord_lanzamiento, coord_destino_inicial,
                        es_centro_indirecto, coord_segundo_lanzamiento,
                        coord_destino_final, velocidad, altura
                    )
                    
                    # Generar participaciones de jugadores (5-10 participaciones)
                    num_participaciones = random.randint(5, 10)
                    
                    for _ in range(num_participaciones):
                        # Determinar si es atacante o defensor
                        es_atacante = random.random() < 0.5
                        
                        if es_atacante:
                            jugador_equipo = equipo_id
                            rol_ofensivo_id = random.choice(rol_ofensivo_ids)
                            rol_defensivo_id = None
                        else:
                            jugador_equipo = visitante_id if equipo_id == local_id else local_id
                            rol_ofensivo_id = None
                            rol_defensivo_id = random.choice(rol_defensivo_ids)
                        
                        # Seleccionar jugador
                        jugadores_disponibles = [j for j in jugadores if j[2] == jugador_equipo and j[0] != lanzador_id]
                        if not jugadores_disponibles:
                            continue
                            
                        jugador_id = random.choice(jugadores_disponibles)[0]
                        
                        # Coordenadas iniciales y finales
                        coord_inicial = f"{random.randint(1, 20)},{random.randint(1, 20)}"
                        coord_final = f"{random.randint(1, 20)},{random.randint(1, 20)}"
                        
                        # Es portero?
                        es_portero = 1 if random.random() < 0.1 else 0
                        
                        # Acción y resultado
                        if es_atacante:
                            acciones = ['Remate de cabeza', 'Remate con el pie', 'Bloqueo', 'Desmarque', 'Arrastre', 'Pase']
                            resultados = ['Exitoso', 'Fallido', 'Interceptado', 'Gol']
                        else:
                            acciones = ['Despeje', 'Anticipación', 'Marcaje', 'Interceptación']
                            resultados = ['Exitoso', 'Fallido', 'Falta']
                        
                        accion = random.choice(acciones)
                        resultado = random.choice(resultados)
                        
                        db.registrar_participacion_jugador(
                            corner_id, jugador_id, jugador_equipo, 
                            rol_ofensivo_id, rol_defensivo_id,
                            coord_inicial, coord_final, 
                            es_portero, accion, resultado
                        )
                    
                    # Registrar resultado del corner
                    tipos_resultado = ['Gol', 'Remate a puerta', 'Remate fuera', 'Despeje', 'Falta en ataque', 
                                     'Falta en defensa', 'Saque de puerta', 'Nuevo corner', 'Continuación de juego']
                    
                    tipo_resultado = random.choice(tipos_resultado)
                    
                    # Determinar si fue gol o remate según el tipo
                    gol = 1 if tipo_resultado == 'Gol' else 0
                    remate = 1 if tipo_resultado in ['Gol', 'Remate a puerta', 'Remate fuera'] else 0
                    
                    # Jugador que remata (si aplica)
                    jugador_remate_id = None
                    if remate == 1:
                        # Buscar un jugador atacante que participó
                        participaciones = db.cursor.execute('''
                        SELECT jugador_id FROM participaciones_jugador 
                        WHERE corner_id = ? AND equipo_id = ? AND rol_ofensivo_id IS NOT NULL
                        ''', (corner_id, equipo_id)).fetchall()
                        
                        if participaciones:
                            jugador_remate_id = random.choice(participaciones)[0]
                    
                    # Valor de xG generado
                    if gol == 1:
                        xg = random.uniform(0.1, 0.5)
                    elif remate == 1:
                        xg = random.uniform(0.05, 0.3)
                    else:
                        xg = random.uniform(0.0, 0.1)
                    
                    # Descripción
                    if gol == 1:
                        descripcion = "Gol tras remate al área"
                    elif remate == 1:
                        if tipo_resultado == 'Remate a puerta':
                            descripcion = "Remate a puerta, parada del portero"
                        else:
                            descripcion = "Remate fuera de los tres palos"
                    else:
                        descripcion = f"Jugada finalizada en {tipo_resultado}"
                    
                    db.registrar_resultado_corner(
                        corner_id, tipo_resultado, gol, remate,
                        jugador_remate_id, xg, descripcion
                    )
        
        db.conn.commit()
        print(f"  - {partidos_generados} partidos generados con {corners_generados} corners")
        
        # Cerrar conexión
        db.close()
        
        return partidos_generados, corners_generados, len(jugador_ids)
    
    except Exception as e:
        print(f"Error al importar datos de muestra: {e}")
        return 0, 0, 0

# Función para limpiar datos
def limpiar_datos():
    """
    Realiza limpieza de datos en la base de datos de corners.
    - Corrige valores inconsistentes
    - Maneja valores nulos o vacíos
    - Estandariza formatos
    """
    try:
        # Conexión a la base de datos
        db = CornerAnalysisDB()
        db.connect()
        
        print("Limpiando datos de la base de datos de corners...")
        
        # 1. Corregir valores inconsistentes en las coordenadas
        # Asegurar que todas las coordenadas tengan el formato correcto "x,y"
        db.cursor.execute('''
        SELECT id, coord_inicial, coord_final 
        FROM participaciones_jugador 
        WHERE coord_inicial IS NOT NULL OR coord_final IS NOT NULL
        ''')
        
        participaciones = db.cursor.fetchall()
        correcciones_coords = 0
        
        for p_id, coord_i, coord_f in participaciones:
            coord_i_fixed = coord_i
            coord_f_fixed = coord_f
            
            # Verificar y corregir coord_inicial
            if coord_i and not coord_i.count(',') == 1:
                # Tratar de corregir
                if ' ' in coord_i:
                    coord_i_fixed = coord_i.replace(' ', ',')
                elif ';' in coord_i:
                    coord_i_fixed = coord_i.replace(';', ',')
                elif '|' in coord_i:
                    coord_i_fixed = coord_i.replace('|', ',')
                
                if coord_i_fixed != coord_i:
                    correcciones_coords += 1
            
            # Verificar y corregir coord_final
            if coord_f and not coord_f.count(',') == 1:
                # Tratar de corregir
                if ' ' in coord_f:
                    coord_f_fixed = coord_f.replace(' ', ',')
                elif ';' in coord_f:
                    coord_f_fixed = coord_f.replace(';', ',')
                elif '|' in coord_f:
                    coord_f_fixed = coord_f.replace('|', ',')
                
                if coord_f_fixed != coord_f:
                    correcciones_coords += 1
            
            # Si hubo correcciones, actualizar el registro
            if coord_i_fixed != coord_i or coord_f_fixed != coord_f:
                db.cursor.execute('''
                UPDATE participaciones_jugador 
                SET coord_inicial = ?, coord_final = ? 
                WHERE id = ?
                ''', (coord_i_fixed, coord_f_fixed, p_id))
        
        # 2. Corrección de valores xG negativos o extremadamente altos
        db.cursor.execute('SELECT id, xg_generado FROM resultados_corner WHERE xg_generado IS NOT NULL')
        resultados = db.cursor.fetchall()
        correcciones_xg = 0
        
        for r_id, xg in resultados:
            if xg < 0:
                # Corregir valores negativos a 0
                db.cursor.execute('UPDATE resultados_corner SET xg_generado = ? WHERE id = ?', (0.0, r_id))
                correcciones_xg += 1
            elif xg > 1:
                # Corregir valores mayores a 1 (las probabilidades van de 0 a 1)
                db.cursor.execute('UPDATE resultados_corner SET xg_generado = ? WHERE id = ?', (1.0, r_id))
                correcciones_xg += 1
        
        # 3. Corregir valores nulos o inconsistentes en roles
        # Verificar participaciones con ambos roles (ofensivo y defensivo) asignados
        db.cursor.execute('''
        SELECT id, rol_ofensivo_id, rol_defensivo_id, equipo_id 
        FROM participaciones_jugador 
        WHERE rol_ofensivo_id IS NOT NULL AND rol_defensivo_id IS NOT NULL
        ''')
        
        participaciones_inconsistentes = db.cursor.fetchall()
        correcciones_roles = 0
        
        for p_id, rol_of, rol_def, equipo_id in participaciones_inconsistentes:
            # Determinar qué rol mantener basado en el equipo y el corner
            db.cursor.execute('''
            SELECT equipo_id FROM corners c 
            JOIN participaciones_jugador p ON c.id = p.corner_id 
            WHERE p.id = ?
            ''', (p_id,))
            
            result = db.cursor.fetchone()
            if result:
                equipo_atacante = result[0]
                
                if equipo_atacante == equipo_id:
                    # Es un atacante, eliminar rol defensivo
                    db.cursor.execute('UPDATE participaciones_jugador SET rol_defensivo_id = NULL WHERE id = ?', (p_id,))
                else:
                    # Es un defensor, eliminar rol ofensivo
                    db.cursor.execute('UPDATE participaciones_jugador SET rol_ofensivo_id = NULL WHERE id = ?', (p_id,))
                
                correcciones_roles += 1
        
        # 4. Verificar y corregir resultados inconsistentes
        # Por ejemplo, remates marcados como gol pero sin gol = 1
        db.cursor.execute('''
        SELECT id, tipo_resultado, gol, remate 
        FROM resultados_corner 
        WHERE (tipo_resultado = 'Gol' AND gol = 0) OR 
              (tipo_resultado LIKE '%Remate%' AND remate = 0) OR
              (tipo_resultado NOT LIKE '%Remate%' AND tipo_resultado != 'Gol' AND (remate = 1 OR gol = 1))
        ''')
        
        resultados_inconsistentes = db.cursor.fetchall()
        correcciones_resultados = 0
        
        for r_id, tipo, gol, remate in resultados_inconsistentes:
            if tipo == 'Gol' and gol == 0:
                db.cursor.execute('UPDATE resultados_corner SET gol = 1 WHERE id = ?', (r_id,))
                correcciones_resultados += 1
            
            if 'Remate' in tipo and remate == 0:
                db.cursor.execute('UPDATE resultados_corner SET remate = 1 WHERE id = ?', (r_id,))
                correcciones_resultados += 1
            
            if 'Remate' not in tipo and tipo != 'Gol' and (remate == 1 or gol == 1):
                db.cursor.execute('UPDATE resultados_corner SET remate = 0, gol = 0 WHERE id = ?', (r_id,))
                correcciones_resultados += 1
        
        # Confirmar cambios
        db.conn.commit()
        
        print(f"  - Coordenadas corregidas: {correcciones_coords}")
        print(f"  - Valores xG corregidos: {correcciones_xg}")
        print(f"  - Roles inconsistentes corregidos: {correcciones_roles}")
        print(f"  - Resultados inconsistentes corregidos: {correcciones_resultados}")
        
        # Cerrar conexión
        db.close()
        
        return correcciones_coords + correcciones_xg + correcciones_roles + correcciones_resultados
    
    except Exception as e:
        print(f"Error al limpiar datos: {e}")
        return 0

# Función para transformar datos
def transformar_datos():
    """
    Realiza transformaciones en los datos para facilitar análisis posteriores.
    - Crea nuevas columnas derivadas
    - Normaliza ciertos valores
    - Genera agregaciones útiles
    """
    try:
        # Conexión a la base de datos
        db = CornerAnalysisDB()
        db.connect()
        
        print("Transformando datos para análisis...")
        
        # 1. Crear vista para análisis de efectividad por equipo
        db.cursor.execute('''
        CREATE VIEW IF NOT EXISTS vista_efectividad_equipos AS
        SELECT 
            e.id AS equipo_id,
            e.nombre AS equipo,
            COUNT(c.id) AS total_corners,
            SUM(CASE WHEN rc.gol = 1 THEN 1 ELSE 0 END) AS goles,
            SUM(CASE WHEN rc.remate = 1 THEN 1 ELSE 0 END) AS remates,
            ROUND(SUM(CASE WHEN rc.gol = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(c.id), 2) AS porcentaje_gol,
            ROUND(SUM(CASE WHEN rc.remate = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(c.id), 2) AS porcentaje_remate,
            ROUND(AVG(rc.xg_generado), 3) AS promedio_xg
        FROM corners c
        JOIN equipos e ON c.equipo_id = e.id
        LEFT JOIN resultados_corner rc ON c.id = rc.corner_id
        GROUP BY e.id
        ORDER BY porcentaje_gol DESC
        ''')
        
        # 2. Crear vista para análisis de efectividad por tipo de lanzamiento
        db.cursor.execute('''
        CREATE VIEW IF NOT EXISTS vista_efectividad_tipos_lanzamiento AS
        SELECT 
            tl.id AS tipo_id,
            tl.nombre AS tipo_lanzamiento,
            COUNT(c.id) AS total_corners,
            SUM(CASE WHEN rc.gol = 1 THEN 1 ELSE'
            SUM(CASE WHEN rc.gol = 1 THEN 1 ELSE 0 END) AS goles,
            SUM(CASE WHEN rc.remate = 1 THEN 1 ELSE 0 END) AS remates,
            ROUND(SUM(CASE WHEN rc.gol = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(c.id), 2) AS porcentaje_gol,
            ROUND(SUM(CASE WHEN rc.remate = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(c.id), 2) AS porcentaje_remate,
            ROUND(AVG(rc.xg_generado), 3) AS promedio_xg
        FROM corners c
        JOIN tipos_lanzamiento tl ON c.tipo_lanzamiento_id = tl.id
        LEFT JOIN resultados_corner rc ON c.id = rc.corner_id
        GROUP BY tl.id
        ORDER BY porcentaje_gol DESC
        ''')
        
        # 3. Crear vista para análisis de roles ofensivos
        db.cursor.execute('''
        CREATE VIEW IF NOT EXISTS vista_efectividad_roles_ofensivos AS
        SELECT 
            ro.id AS rol_id,
            ro.nombre AS rol_ofensivo,
            COUNT(DISTINCT pj.id) AS total_participaciones,
            SUM(CASE WHEN rc.gol = 1 THEN 1 ELSE 0 END) AS goles_participados,
            SUM(CASE WHEN rc.remate = 1 THEN 1 ELSE 0 END) AS remates_participados,
            ROUND(SUM(CASE WHEN rc.gol = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(DISTINCT pj.id), 2) AS ratio_gol,
            ROUND(AVG(rc.xg_generado), 3) AS promedio_xg
        FROM participaciones_jugador pj
        JOIN roles_ofensivos ro ON pj.rol_ofensivo_id = ro.id
        JOIN corners c ON pj.corner_id = c.id
        LEFT JOIN resultados_corner rc ON c.id = rc.corner_id
        WHERE pj.rol_ofensivo_id IS NOT NULL
        GROUP BY ro.id
        ''')
        
        # 4. Crear tabla derivada de distancias recorridas por cada jugador en corners
        db.cursor.execute('''
        CREATE TABLE IF NOT EXISTS distancia_movimientos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            participacion_id INTEGER,
            jugador_id INTEGER,
            corner_id INTEGER,
            distancia FLOAT,
            FOREIGN KEY (participacion_id) REFERENCES participaciones_jugador (id),
            FOREIGN KEY (jugador_id) REFERENCES jugadores (id),
            FOREIGN KEY (corner_id) REFERENCES corners (id)
        )
        ''')
        
        # Calcular distancias para participaciones que tienen coordenadas iniciales y finales
        db.cursor.execute('''
        SELECT id, jugador_id, corner_id, coord_inicial, coord_final 
        FROM participaciones_jugador 
        WHERE coord_inicial IS NOT NULL AND coord_final IS NOT NULL
        ''')
        
        participaciones = db.cursor.fetchall()
        distancias_calculadas = 0
        
        # Limpiamos la tabla para recalcular
        db.cursor.execute('DELETE FROM distancia_movimientos')
        
        for p_id, jugador_id, corner_id, coord_i, coord_f in participaciones:
            try:
                # Extraer coordenadas
                x1, y1 = map(float, coord_i.split(','))
                x2, y2 = map(float, coord_f.split(','))
                
                # Calcular distancia euclidiana
                distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                
                # Guardar en tabla derivada
                db.cursor.execute('''
                INSERT INTO distancia_movimientos (participacion_id, jugador_id, corner_id, distancia)
                VALUES (?, ?, ?, ?)
                ''', (p_id, jugador_id, corner_id, distancia))
                
                distancias_calculadas += 1
            except:
                # Si hay errores en el formato de las coordenadas, saltamos
                continue
        
        # 5. Crear tabla derivada para clasificar la peligrosidad de los corners
        db.cursor.execute('''
        CREATE TABLE IF NOT EXISTS clasificacion_peligrosidad (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            corner_id INTEGER,
            nivel_peligrosidad TEXT,
            score FLOAT,
            FOREIGN KEY (corner_id) REFERENCES corners (id)
        )
        ''')
        
        # Calcular peligrosidad basada en xG y resultado
        db.cursor.execute('''
        SELECT c.id, rc.xg_generado, rc.gol, rc.remate
        FROM corners c
        JOIN resultados_corner rc ON c.id = rc.corner_id
        ''')
        
        corners = db.cursor.fetchall()
        
        # Limpiamos la tabla para recalcular
        db.cursor.execute('DELETE FROM clasificacion_peligrosidad')
        
        clasificaciones_generadas = 0
        
        for corner_id, xg, gol, remate in corners:
            # Calcular score de peligrosidad (combinación ponderada de factores)
            xg = xg or 0  # Manejar NULL
            score = xg * 0.6 + (gol * 0.3) + (remate * 0.1)
            
            # Clasificar según score
            if score >= 0.5:
                nivel = "Alta"
            elif score >= 0.2:
                nivel = "Media"
            else:
                nivel = "Baja"
            
            # Guardar clasificación
            db.cursor.execute('''
            INSERT INTO clasificacion_peligrosidad (corner_id, nivel_peligrosidad, score)
            VALUES (?, ?, ?)
            ''', (corner_id, nivel, score))
            
            clasificaciones_generadas += 1
        
        # Confirmar cambios
        db.conn.commit()
        
        print(f"  - Se crearon 3 vistas para análisis de efectividad")
        print(f"  - Se calcularon distancias para {distancias_calculadas} participaciones")
        print(f"  - Se generaron {clasificaciones_generadas} clasificaciones de peligrosidad")
        
        # Cerrar conexión
        db.close()
        
        return distancias_calculadas + clasificaciones_generadas
    
    except Exception as e:
        print(f"Error al transformar datos: {e}")
        return 0

# Demostración del proceso de limpieza y transformación
def demostrar_limpieza_y_transformacion():
    """
    Demuestra el proceso completo de importación, limpieza y transformación de datos.
    """
    print("\n=== DEMOSTRACIÓN DE LIMPIEZA Y PREPROCESAMIENTO DE DATOS ===\n")
    
    # Paso 1: Importar datos de muestra
    print("Paso 1: Importación de datos de muestra")
    partidos, corners, jugadores = importar_datos_muestra()
    print(f"Datos importados: {partidos} partidos, {corners} corners, {jugadores} jugadores\n")
    
    # Paso 2: Introducir algunos datos "sucios" para demostrar limpieza
    print("Paso 2: Introducción de datos inconsistentes para demostración")
    try:
        db = CornerAnalysisDB()
        db.connect()
        
        # Introducir coordenadas mal formateadas
        db.cursor.execute('''
        UPDATE participaciones_jugador 
        SET coord_inicial = '10 15', coord_final = '5|8'
        WHERE id IN (SELECT id FROM participaciones_jugador LIMIT 5)
        ''')
        
        # Introducir xG negativos o extremos
        db.cursor.execute('''
        UPDATE resultados_corner 
        SET xg_generado = -0.1
        WHERE id IN (SELECT id FROM resultados_corner LIMIT 3)
        ''')
        
        db.cursor.execute('''
        UPDATE resultados_corner 
        SET xg_generado = 1.5
        WHERE id IN (SELECT id FROM resultados_corner LIMIT 3 OFFSET 3)
        ''')
        
        # Introducir inconsistencias en roles
        db.cursor.execute('''
        UPDATE participaciones_jugador 
        SET rol_ofensivo_id = 1, rol_defensivo_id = 1
        WHERE id IN (SELECT id FROM participaciones_jugador LIMIT 5 OFFSET 10)
        ''')
        
        # Introducir inconsistencias en resultados
        db.cursor.execute('''
        UPDATE resultados_corner 
        SET tipo_resultado = 'Gol', gol = 0
        WHERE id IN (SELECT id FROM resultados_corner LIMIT 2 OFFSET 6)
        ''')
        
        db.cursor.execute('''
        UPDATE resultados_corner 
        SET tipo_resultado = 'Remate a puerta', remate = 0
        WHERE id IN (SELECT id FROM resultados_corner LIMIT 2 OFFSET 8)
        ''')
        
        db.conn.commit()
        db.close()
        print("Datos inconsistentes introducidos correctamente para demostración")
    except Exception as e:
        print(f"Error al introducir datos inconsistentes: {e}")
    
    # Paso 3: Limpieza de datos
    print("\nPaso 3: Limpieza de datos")
    num_correcciones = limpiar_datos()
    print(f"Total de correcciones aplicadas: {num_correcciones}\n")
    
    # Paso 4: Transformación de datos
    print("Paso 4: Transformación de datos")
    num_transformaciones = transformar_datos()
    print(f"Total de transformaciones aplicadas: {num_transformaciones}\n")
    
    # Paso 5: Mostrar algunos resultados
    print("Paso 5: Verificación de resultados")
    try:
        db = CornerAnalysisDB()
        db.connect()
        
        # Mostrar efectividad por equipos
        print("\nEfectividad por equipos:")
        db.cursor.execute('SELECT equipo, total_corners, goles, porcentaje_gol, promedio_xg FROM vista_efectividad_equipos LIMIT 3')
        for row in db.cursor.fetchall():
            print(f"  - {row[0]}: {row[1]} corners, {row[2]} goles ({row[3]}%), xG promedio: {row[4]}")
        
        # Mostrar efectividad por tipos de lanzamiento
        print("\nEfectividad por tipos de lanzamiento:")
        db.cursor.execute('SELECT tipo_lanzamiento, total_corners, goles, porcentaje_gol FROM vista_efectividad_tipos_lanzamiento LIMIT 3')
        for row in db.cursor.fetchall():
            print(f"  - {row[0]}: {row[1]} corners, {row[2]} goles ({row[3]}%)")
        
        # Mostrar distancias promedio por rol
        print("\nDistancias promedio por rol ofensivo:")
        db.cursor.execute('''
        SELECT ro.nombre, ROUND(AVG(dm.distancia), 2) AS distancia_promedio
        FROM distancia_movimientos dm
        JOIN participaciones_jugador pj ON dm.participacion_id = pj.id
        JOIN roles_ofensivos ro ON pj.rol_ofensivo_id = ro.id
        WHERE pj.rol_ofensivo_id IS NOT NULL
        GROUP BY ro.nombre
        ORDER BY distancia_promedio DESC
        LIMIT 5
        ''')
        for row in db.cursor.fetchall():
            print(f"  - {row[0]}: {row[1]} unidades")
        
        # Mostrar clasificación de peligrosidad
        print("\nDistribución de peligrosidad de corners:")
        db.cursor.execute('''
        SELECT nivel_peligrosidad, COUNT(*) AS cantidad, ROUND(AVG(score), 3) AS score_promedio
        FROM clasificacion_peligrosidad
        GROUP BY nivel_peligrosidad
        ORDER BY score_promedio DESC
        ''')
        for row in db.cursor.fetchall():
            print(f"  - {row[0]}: {row[1]} corners (score promedio: {row[2]})")
        
        db.close()
    except Exception as e:
        print(f"Error al mostrar resultados: {e}")
    
    print("\n=== FIN DE LA DEMOSTRACIÓN ===")

# Ejecutar la demostración
demostrar_limpieza_y_transformacion()


=== DEMOSTRACIÓN DE LIMPIEZA Y PREPROCESAMIENTO DE DATOS ===

Paso 1: Importación de datos de muestra
Conexión establecida con corner_analysis.db
Importando datos de muestra para análisis de corners...
  - 5 competiciones importadas
  - 8 equipos importados
Error al importar datos de muestra: name 'timedelta' is not defined
Datos importados: 0 partidos, 0 corners, 0 jugadores

Paso 2: Introducción de datos inconsistentes para demostración
Conexión establecida con corner_analysis.db
Conexión cerrada
Datos inconsistentes introducidos correctamente para demostración

Paso 3: Limpieza de datos
Conexión establecida con corner_analysis.db
Limpiando datos de la base de datos de corners...
  - Coordenadas corregidas: 4
  - Valores xG corregidos: 2
  - Roles inconsistentes corregidos: 0
  - Resultados inconsistentes corregidos: 0
Conexión cerrada
Total de correcciones aplicadas: 6

Paso 4: Transformación de datos
Conexión establecida con corner_analysis.db
Transformando datos para análisis...
